In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from scipy.stats import randint, uniform
from imblearn.over_sampling import SMOTE  # 用于处理类别不平衡

# 1. 加载数据（这里使用示例数据，实际应用时替换为自己的数据）
# 示例数据生成 - 多分类问题（3个类别）
np.random.seed(42)
X = np.random.rand(1000, 10)  # 1000个样本，10个特征
y = np.random.choice([0, 1, 2], size=1000, p=[0.7, 0.2, 0.1])  # 不平衡的多分类标签

# 检查类别分布
print("类别分布:", np.bincount(y))

# 划分训练集和测试集（使用分层分割保持类别比例）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 2. 处理类别不平衡（可选）
print("\n应用SMOTE过采样...")
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样后类别分布:", np.bincount(y_train_res))

# 3. 创建基础随机森林分类器
rf = RandomForestClassifier(random_state=42, class_weight='balanced')

# 4. 设置参数分布用于随机搜索
param_dist = {
    'n_estimators': randint(50, 500),  # 树的数量
    'max_depth': [None] + list(np.arange(5, 30)),  # 树的最大深度
    'min_samples_split': randint(2, 20),  # 分裂内部节点所需的最小样本数
    'min_samples_leaf': randint(1, 20),  # 叶节点所需的最小样本数
    'max_features': ['sqrt', 'log2', None] + list(np.linspace(0.1, 1.0, 10)),  # 考虑的特征数量
    'bootstrap': [True, False],  # 是否使用bootstrap样本
    'criterion': ['gini', 'entropy'],  # 分裂标准
    'max_samples': [None] + list(np.linspace(0.1, 1.0, 10))  # bootstrap样本比例
}

# 5. 设置分层交叉验证（保持每折的类别分布）
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 6. 设置随机搜索
# 使用macro-F1作为主要评估指标（平等看待所有类别）
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=100,  # 尝试100组不同的参数
    cv=cv,  # 分层5折交叉验证
    scoring='f1_macro',  # 多分类使用macro-F1
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# 7. 在训练数据上执行随机搜索（使用过采样后的数据）
print("\n开始随机搜索调参...")
random_search.fit(X_train_res, y_train_res)

# 8. 输出最佳参数和最佳得分
print("\n最佳参数组合:")
print(random_search.best_params_)
print(f"\n最佳交叉验证分数(macro-F1): {random_search.best_score_:.4f}")

# 9. 使用最佳模型进行预测
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)

# 10. 评估测试集性能
print("\n测试集性能:")
print(f"准确率: {accuracy_score(y_test, y_pred):.4f}")
print(f"Macro-F1: {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Weighted-F1: {f1_score(y_test, y_pred, average='weighted'):.4f}")

# 11. 输出分类报告和混淆矩阵
print("\n分类报告:")
print(classification_report(y_test, y_pred))

print("\n混淆矩阵:")
print(confusion_matrix(y_test, y_pred))

# 12. 可视化特征重要性
import matplotlib.pyplot as plt

feature_importance = best_rf.feature_importances_
sorted_idx = np.argsort(feature_importance)[::-1]

plt.figure(figsize=(10, 6))
plt.bar(range(X.shape[1]), feature_importance[sorted_idx], align='center')
plt.xticks(range(X.shape[1]), sorted_idx)
plt.xlabel('特征索引')
plt.ylabel('特征重要性')
plt.title('随机森林特征重要性')
plt.show()